In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

Załadowanie plików źródłowych:


*   Property_sales_data_New_York.csv
*   Airbnb_data_New_York.csv



In [2]:
sales_NY = pd.read_csv('Property_sales_data_New_York.csv')
airbnb_NY = pd.read_csv('Airbnb_data_New_York.csv')

Uporządkowanie ramki danych o sprzedaży mieszkań w Nowym Jorku - sales_NY



In [3]:
# Usunięcie zbędnych kolumn
try:
    sales_NY = sales_NY.drop(columns=['Unnamed: 0'])
except:
    pass

# Usunięcie zduplikwowanych wierszy
sales_NY.drop_duplicates()

# Dodanie nowej kolumny z dzielnicą
brough_names = {1: "Manhattan",
2: "Bronx",
3: "Brooklyn",
4: "Queens",
5: "Staten Island"}

for i in list(brough_names.keys()):
    sales_NY.loc[sales_NY["BOROUGH"] == i, "BOROUGH NAME"] = brough_names[i]

# Modyfikacja kolumn tekstowych na numeryczne
sales_NY["SALE PRICE"] = sales_NY["SALE PRICE"].str.replace('\D', "", regex=True)
sales_NY = sales_NY[sales_NY["SALE PRICE"] != ""]
sales_NY = sales_NY.reset_index()

sales_NY["GROSS SQUARE FEET"] = sales_NY["GROSS SQUARE FEET"].str.replace('\D', "", regex=True)
sales_NY = sales_NY[np.logical_and(sales_NY["GROSS SQUARE FEET"] != "", sales_NY["GROSS SQUARE FEET"] != "0")]
sales_NY = sales_NY.reset_index()
sales_NY = sales_NY.drop(columns=['level_0'])

sales_NY["LAND SQUARE FEET"] = sales_NY["LAND SQUARE FEET"].str.replace('\D', "", regex=True)
sales_NY = sales_NY[np.logical_and(sales_NY["LAND SQUARE FEET"] != "", sales_NY["LAND SQUARE FEET"] != "0")]

sales_NY = sales_NY.reset_index()
try:
    sales_NY = sales_NY.drop(columns=['level_0'])
except:
    pass

for column in list(sales_NY.columns):
    try:
        sales_NY[column] = sales_NY[column].apply(pd.to_numeric)
    except:
        pass

# Odfiltrowanie niewłaściwych wartości ceny równej 0
sales_NY = sales_NY[sales_NY["SALE PRICE"] > 0]
sales_NY = sales_NY.reset_index()

# Dodanie kolumny z ceną stopy kwadratowej ("PRICE PER SQUARE FT")
sales_NY["PRICE PER SQUARE FT"] = round(sales_NY["SALE PRICE"] / sales_NY["GROSS SQUARE FEET"],2)

# Dodanie kolumny z dzielnicami do dalszego porównania
sales_NY["Neighborhood"] = sales_NY["NEIGHBORHOOD"].astype(str)
sales_NY["Neighborhood"] = sales_NY["Neighborhood"].str.replace("'", "", regex=True)
sales_NY["Neighborhood"] = sales_NY["Neighborhood"].str.replace("-", " ", regex=True)

# Usunięcie zbędnych kolumn
try:
    sales_NY = sales_NY.drop(columns=['level_0','index'])
except:
    pass

sales_NY = sales_NY.reset_index()

sales_NY.head(10)

,index,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,BOROUGH NAME,PRICE PER SQUARE FT,Neighborhood
0,0,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00,Manhattan,1028.73,ALPHABET CITY
1,1,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00,Manhattan,579.37,ALPHABET CITY
2,2,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00,Manhattan,1733.48,ALPHABET CITY
3,3,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,406,32,,C4,210 AVENUE B,...,1750,4226,1920,2,C4,3192840,2016-09-23 00:00:00,Manhattan,755.52,ALPHABET CITY
4,4,1,ALPHABET CITY,08 RENTALS - ELEVATOR APARTMENTS,2,387,153,,D9,629 EAST 5TH STREET,...,4489,18523,1920,2,D9,16232000,2016-11-07 00:00:00,Manhattan,876.32,ALPHABET CITY
5,5,1,ALPHABET CITY,08 RENTALS - ELEVATOR APARTMENTS,2B,400,21,,D1,530-532 EAST 5TH STREET,...,3717,12350,2009,2,D1,10350000,2016-10-17 00:00:00,Manhattan,838.06,ALPHABET CITY
6,6,1,ALPHABET CITY,09 COOPS - WALKUP APARTMENTS,2,376,14,,C6,244 EAST 7TH STREET,...,4131,16776,1928,2,C6,11900000,2017-06-21 00:00:00,Manhattan,709.35,ALPHABET CITY
7,7,1,ALPHABET CITY,14 RENTALS - 4-10 UNIT,2A,391,19,,S3,624 EAST 9TH STREET,...,1520,3360,1910,2,S3,3300000,2016-11-15 00:00:00,Manhattan,982.14,ALPHABET CITY
8,8,1,ALPHABET CITY,14 RENTALS - 4-10 UNIT,2A,393,4,,S4,165 AVENUE B,...,2201,5608,1900,2,S4,7215000,2017-01-30 00:00:00,Manhattan,1286.55,ALPHABET CITY
9,9,1,ALPHABET CITY,14 RENTALS - 4-10 UNIT,2A,394,5,,S5,183 AVENUE B,...,1779,3713,1910,2,S5,4750000,2017-04-03 00:00:00,Manhattan,1279.29,ALPHABET CITY


Uporządkowanie ramki danych o wynajmie krótkoterminowym w Nowym Jorku - airbnb_NY

In [4]:
# Usunięcie zduplikwowanych wierszy
airbnb_NY.drop_duplicates()

# Odfiltrowanie zerowych wartości
airbnb_NY = airbnb_NY[np.logical_and(airbnb_NY["availability_365"] > 0, airbnb_NY["price"] > 0)]
airbnb_NY = airbnb_NY.reset_index()

# Uporządkowanie kolumny z dzielnicą
airbnb_NY["Neighborhood"] = airbnb_NY['neighbourhood'].astype(str)
airbnb_NY["Neighborhood"] = airbnb_NY["Neighborhood"].str.replace("'", "", regex=True)
airbnb_NY["Neighborhood"] = airbnb_NY["Neighborhood"].str.replace("-", " ", regex=True)
airbnb_NY['Neighborhood'] = airbnb_NY['Neighborhood'].str.upper()

# Usunięcie zbędnych kolumn
try:
    airbnb_NY = airbnb_NY.drop(columns=['level_0','index'])
except:
    pass
airbnb_NY = airbnb_NY.reset_index()

airbnb_NY.head(10)

,level_0,index,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,Neighborhood
0,0,0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,KENSINGTON
1,1,1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,MIDTOWN
2,2,2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365,HARLEM
3,3,3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194,CLINTON HILL
4,4,5,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,2019-06-22,0.59,1,129,MURRAY HILL
5,5,7,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79,2,430,2019-06-24,3.47,1,220,HELLS KITCHEN
6,6,9,5238,Cute & Cozy Lower East Side 1 bdrm,7549,Ben,Manhattan,Chinatown,40.71344,-73.99037,Entire home/apt,150,1,160,2019-06-09,1.33,4,188,CHINATOWN
7,7,10,5295,Beautiful 1br on Upper West Side,7702,Lena,Manhattan,Upper West Side,40.80316,-73.96545,Entire home/apt,135,5,53,2019-06-22,0.43,1,6,UPPER WEST SIDE
8,8,11,5441,Central Manhattan/near Broadway,7989,Kate,Manhattan,Hell's Kitchen,40.76076,-73.98867,Private room,85,2,188,2019-06-23,1.50,1,39,HELLS KITCHEN
9,9,12,5803,"Lovely Room 1, Garden, Best Area, Legal rental",9744,Laurie,Brooklyn,South Slope,40.66829,-73.98779,Private room,89,4,167,2019-06-24,1.34,3,314,SOUTH SLOPE


Ujednolicenie danych w kolumnach z dzielnicami w obu ramkach danych, żeby było możliwe ich porównanie

In [5]:
Neighborhood_dict = {'BAY TERRACE' : 'BAY TERRACE, STATEN ISLAND',
             'CASTLE HILL' : 'CASTLE HILL/UNIONPORT',
             'UNIONPORT' : 'CASTLE HILL/UNIONPORT',
             'FLATBUSH CENTRAL' : 'FLATBUSH',
             'FLATBUSH EAST' : 'EAST FLATBUSH',
             'FLATBUSH LEFFERTS GARDEN' : 'FLATBUSH',
             'FLATBUSH NORTH' : 'FLATBUSH',
             'FLUSHING NORTH' : 'FLUSHING',
             'FLUSHING SOUTH' : 'FLUSHING',
             'FLATIRON DISTRICT' : 'FLATIRON',
             'FINANCIAL DISTRICT' : 'FINANCIAL',
             'GREENWICH VILLAGE CENTRAL' : 'GREENWICH VILLAGE',
             'GREENWICH VILLAGE WEST' : 'GREENWICH VILLAGE',
             'EAST HARLEM' : 'HARLEM EAST',
             'HARLEM CENTRAL' : 'HARLEM',
             'HARLEM UPPER' : 'HARLEM',
             'HARLEM WEST' : 'HARLEM',
             'HIGHBRIDGE' : 'HIGHBRIDGE/MORRIS HEIGHTS',
             'MORRIS HEIGHTS' : 'HIGHBRIDGE/MORRIS HEIGHTS',
             'KINGSBRIDGE' : 'KINGSBRIDGE/JEROME PARK',
             'MELROSE' : 'MELROSE/CONCOURSE',
             'CONCOURSE' : 'MELROSE/CONCOURSE',
             'MIDTOWN CBD' : 'MIDTOWN',
             'MIDTOWN EAST' : 'MIDTOWN',
             'MIDTOWN WEST'  : 'MIDTOWN',
             'VAN NEST' : 'MORRIS PARK/VAN NEST',
             'MORRIS PARK' : 'MORRIS PARK/VAN NEST',
             'MOUNT HOPE' : 'MOUNT HOPE/MOUNT EDEN',
             'MOUNT EDEN' : 'MOUNT HOPE/MOUNT EDEN',
             'MORRISANIA' : 'MORRISANIA/LONGWOOD',
             'LONGWOOD' : 'MORRISANIA/LONGWOOD',
             'MOTT HAVEN' : 'MOTT HAVEN/PORT MORRIS',
             'PORT MORRIS' : 'MOTT HAVEN/PORT MORRIS',
             'SCHUYLERVILLE' : 'SCHUYLERVILLE/PELHAM BAY',
             'PELHAM BAY' : 'SCHUYLERVILLE/PELHAM BAY',
             'UPPER EAST SIDE (59 79)' : 'UPPER EAST SIDE',
             'UPPER EAST SIDE (79 96)' : 'UPPER EAST SIDE',
             'UPPER EAST SIDE (96 110)' : 'UPPER EAST SIDE',
             'UPPER WEST SIDE (59 79)' : 'UPPER WEST SIDE',
             'UPPER WEST SIDE (79 96)' : 'UPPER WEST SIDE',
             'UPPER WEST SIDE (96 116)' : 'UPPER WEST SIDE',
             'WASHINGTON HEIGHTS LOWER' : 'WASHINGTON HEIGHTS',
             'WASHINGTON HEIGHTS UPPER' : 'WASHINGTON HEIGHTS',
             'WILLIAMSBURG CENTRAL' : 'WILLIAMSBURG',
             'WILLIAMSBURG EAST' : 'WILLIAMSBURG',
             'WILLIAMSBURG NORTH' : 'WILLIAMSBURG',
             'WILLIAMSBURG SOUTH' : 'WILLIAMSBURG'
            }

for i in list(Neighborhood_dict.keys()):
    if i in set(sales_NY["Neighborhood"]):
        sales_NY.loc[sales_NY["Neighborhood"] == i, "Neighborhood"] = Neighborhood_dict[i]
    elif i in set(airbnb_NY["Neighborhood"]):
        airbnb_NY.loc[airbnb_NY["Neighborhood"] == i, "Neighborhood"] = Neighborhood_dict[i]

Neighborhoods_list = list(set(sales_NY["Neighborhood"]).intersection(set(airbnb_NY['Neighborhood'])))
Neighborhoods_list.sort()

# Select only wanted columns in sales_NY and fix format of columns names and data in columns
sales_NY = sales_NY[['BOROUGH NAME', 'Neighborhood', 'BUILDING CLASS CATEGORY',
       'TAX CLASS AT PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING CLASS AT PRESENT', 'ADDRESS', 'APARTMENT NUMBER', 'ZIP CODE',
       'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',
       'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
       'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE',
       'SALE PRICE', 'SALE DATE', 'PRICE PER SQUARE FT']].copy()

sales_NY = sales_NY.rename(str.title, axis='columns')
sales_NY = sales_NY.rename(columns = {"Borough Name" : "Borough"})
sales_NY['Address'] = sales_NY['Address'].str.title()
sales_NY['Building Class Category'] = sales_NY['Building Class Category'].str.title()
sales_NY["Sale Date"] = pd.to_datetime(sales_NY["Sale Date"])

for i in range(0, len(sales_NY["Neighborhood"])-1):
    if sales_NY["Neighborhood"][i] not in Neighborhoods_list:
        sales_NY = sales_NY.drop(index = i)
sales_NY = sales_NY.reset_index()
sales_NY['Neighborhood'] = sales_NY['Neighborhood'].str.title()
sales_NY = sales_NY.drop(columns=['index'])

# Select only wanted columns in airbnb_NY and fix format of columns names and data in columns
airbnb_NY = airbnb_NY[['Neighborhood', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365']].copy()

airbnb_NY = airbnb_NY.rename(columns = {"neighbourhood_group" : "Borough"})
airbnb_NY.columns = airbnb_NY.columns.str.replace("_", " ", regex=True).str.title()
for i in range(0, len(airbnb_NY['Neighborhood'])-1):
    if airbnb_NY['Neighborhood'][i] not in Neighborhoods_list:
        airbnb_NY = airbnb_NY.drop(index = i)
airbnb_NY = airbnb_NY.reset_index()
airbnb_NY['Neighborhood'] = airbnb_NY['Neighborhood'].str.title()
airbnb_NY = airbnb_NY.drop(index=airbnb_NY.loc[airbnb_NY["Neighborhood"] == "Hells Kitchen"].index)

airbnb_NY = airbnb_NY.drop(columns=['index'])

Statystyki tabeli sales_NY

In [6]:
# Sprawdzenie, czy wszystkie niechciane puste komórki zostały usunięte z sales_NY
sales_NY.isnull().sum()

Borough                           0
Neighborhood                      0
Building Class Category           0
Tax Class At Present              0
Block                             0
Lot                               0
Ease-Ment                         0
Building Class At Present         0
Address                           0
Apartment Number                  0
Zip Code                          0
Residential Units                 0
Commercial Units                  0
Total Units                       0
Land Square Feet                  0
Gross Square Feet                 0
Year Built                        0
Tax Class At Time Of Sale         0
Building Class At Time Of Sale    0
Sale Price                        0
Sale Date                         0
Price Per Square Ft               0
dtype: int64

In [7]:
# Podsumowanie każdej kolumny z sales_NY
sales_NY.describe()

,Block,Lot,Zip Code,Residential Units,Commercial Units,Total Units,Land Square Feet,Gross Square Feet,Year Built,Tax Class At Time Of Sale,Sale Price,Price Per Square Ft
count,24675.000000,24675.000000,24675.000000,24675.000000,24675.000000,24675.000000,2.467500e+04,2.467500e+04,24675.000000,24675.000000,2.467500e+04,24675.000000
mean,5437.718014,58.127579,11010.330902,3.125593,0.362472,3.486525,4.144195e+03,4.638010e+03,1939.263506,1.242148,1.668936e+06,415.692107
std,3725.996844,120.684509,516.093363,17.772211,15.272189,23.522738,2.763450e+04,2.726120e+04,49.313030,0.701674,1.776133e+07,675.312163
min,10.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000e+02,1.200000e+02,0.000000,1.000000,1.000000e+00,0.000000
25%,2510.500000,19.000000,10466.000000,1.000000,0.000000,1.000000,2.000000e+03,1.391000e+03,1920.000000,1.000000,4.250000e+05,234.680000
50%,4826.000000,37.000000,11221.000000,2.000000,0.000000,2.000000,2.500000e+03,1.920000e+03,1930.000000,1.000000,6.400000e+05,346.980000
75%,7595.000000,62.000000,11374.000000,2.000000,0.000000,2.000000,3.948500e+03,2.731000e+03,1955.000000,1.000000,9.850000e+05,495.425000
max,16319.000000,7501.000000,11694.000000,948.000000,2261.000000,2261.000000,2.128899e+06,1.617206e+06,2017.000000,4.000000,2.210000e+09,75974.860000


In [8]:
sales_NY.head(10)

,Borough,Neighborhood,Building Class Category,Tax Class At Present,Block,Lot,Ease-Ment,Building Class At Present,Address,Apartment Number,...,Commercial Units,Total Units,Land Square Feet,Gross Square Feet,Year Built,Tax Class At Time Of Sale,Building Class At Time Of Sale,Sale Price,Sale Date,Price Per Square Ft
0,Manhattan,Chelsea,01 One Family Dwellings,1,771,52,,A9,224 West 22Nd Street,,...,0,1,1975,3855,1901,1,A9,7425000,2016-12-30,1926.07
1,Manhattan,Chelsea,01 One Family Dwellings,1,772,29,,A4,231 West 22Nd Street,,...,0,1,1588,5653,1900,1,A4,10,2017-01-30,0.00
2,Manhattan,Chelsea,01 One Family Dwellings,1,772,29,,A4,231 West 22Nd Street,,...,0,1,1588,5653,1900,1,A4,10,2017-01-30,0.00
3,Manhattan,Chelsea,01 One Family Dwellings,1,772,29,,A4,231 West 22Nd Street,,...,0,1,1588,5653,1900,1,A4,10,2017-01-30,0.00
4,Manhattan,Chelsea,02 Two Family Dwellings,1,769,49,,S2,218 West 20Th Street,,...,1,3,2150,4978,1920,1,S2,7750000,2017-04-17,1556.85
5,Manhattan,Chelsea,03 Three Family Dwellings,1,719,67,,C0,444 West 22Nd Street,,...,0,3,2469,3725,1901,1,C0,8300000,2017-02-02,2228.19
6,Manhattan,Chelsea,07 Rentals - Walkup Apartments,2B,696,37,,C4,253 10Th Avenue,,...,1,7,1463,3762,1901,2,C4,16000000,2016-09-08,4253.06
7,Manhattan,Chelsea,07 Rentals - Walkup Apartments,2A,722,6,,C3,459 West 24Th Street,,...,0,4,2057,2890,1910,2,C3,5950000,2016-10-26,2058.82
8,Manhattan,Chelsea,07 Rentals - Walkup Apartments,2,743,56,,C5,318 West 20Th Street,,...,0,24,2307,4500,1900,2,C5,7375000,2017-02-06,1638.89
9,Manhattan,Chelsea,07 Rentals - Walkup Apartments,2A,745,63,,C3,348 West 22Nd Street,,...,0,4,2057,3042,1900,2,C3,8000000,2017-02-06,2629.85


Statystyki tabeli airbnb_NY

In [9]:
# Sprawdzenie, czy wszystkie niechciane puste komórki zostały usunięte z airbnb_NY
airbnb_NY.isnull().sum()

Neighborhood                         0
Name                                 3
Host Id                              0
Host Name                            8
Borough                              0
Latitude                             0
Longitude                            0
Room Type                            0
Price                                0
Minimum Nights                       0
Number Of Reviews                    0
Last Review                       4495
Reviews Per Month                 4495
Calculated Host Listings Count       0
Availability 365                     0
dtype: int64

In [10]:
# Podsumowanie każdej kolumny z airbnb_NY
airbnb_NY.describe()

,Host Id,Latitude,Longitude,Price,Minimum Nights,Number Of Reviews,Reviews Per Month,Calculated Host Listings Count,Availability 365
count,2.754200e+04,27542.000000,27542.000000,27542.000000,27542.000000,27542.000000,23047.000000,27542.000000,27542.000000
mean,8.025795e+07,40.726817,-73.945104,157.329388,8.265485,31.964418,1.795678,9.847760,175.484569
std,8.589854e+07,0.056431,0.050523,251.028504,23.686787,51.770361,1.754282,39.773399,126.097016
min,2.571000e+03,40.499790,-74.244420,10.000000,1.000000,0.000000,0.010000,1.000000,1.000000
25%,8.503180e+06,40.687730,-73.976710,69.000000,2.000000,2.000000,0.450000,1.000000,55.000000
50%,4.029385e+07,40.719360,-73.951235,109.000000,3.000000,10.000000,1.230000,1.000000,167.000000
75%,1.408304e+08,40.761230,-73.928253,180.000000,5.000000,39.000000,2.660000,3.000000,304.000000
max,2.743115e+08,40.904840,-73.712990,10000.000000,1250.000000,629.000000,20.940000,327.000000,365.000000


In [11]:
airbnb_NY.head(10)

,Neighborhood,Name,Host Id,Host Name,Borough,Latitude,Longitude,Room Type,Price,Minimum Nights,Number Of Reviews,Last Review,Reviews Per Month,Calculated Host Listings Count,Availability 365
0,Kensington,Clean & quiet apt home by the park,2787,John,Brooklyn,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,Midtown,Skylit Midtown Castle,2845,Jennifer,Manhattan,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,Harlem,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,Clinton Hill,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,Murray Hill,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,40.74767,-73.97500,Entire home/apt,200,3,74,2019-06-22,0.59,1,129
5,Chinatown,Cute & Cozy Lower East Side 1 bdrm,7549,Ben,Manhattan,40.71344,-73.99037,Entire home/apt,150,1,160,2019-06-09,1.33,4,188
6,Upper West Side,Beautiful 1br on Upper West Side,7702,Lena,Manhattan,40.80316,-73.96545,Entire home/apt,135,5,53,2019-06-22,0.43,1,6
7,Upper West Side,Wonderful Guest Bedroom in Manhattan for SINGLES,11528,Claudio,Manhattan,40.79826,-73.96113,Private room,85,2,113,2019-07-05,0.91,1,333
8,Williamsburg,Only 2 stops to Manhattan studio,15991,Allen & Irina,Brooklyn,40.70837,-73.95352,Entire home/apt,140,2,148,2019-06-29,1.20,1,46
9,Fort Greene,Perfect for Your Parents + Garden,17571,Jane,Brooklyn,40.69169,-73.97185,Entire home/apt,215,2,198,2019-06-28,1.72,1,321


In [12]:
# Zapis wyczyszczonych tabeli jako .json do użycia w aplikacji Case_study_app.py
sales_NY.to_json('sales_NY.json')
airbnb_NY.to_json('airbnb_NY.json')